In [1]:
import os
import xml.etree.ElementTree as ET

def create_ttl_relations(input_filename, output_filename):
    # Define the namespace and other prefixes
    prefixes = {
        'ex': 'http://example.org/',
    }

    base_uri = prefixes['ex']

    # Parse the XML file
    tree = ET.parse(input_filename)
    root = tree.getroot()

    unique_keys = set()

    # Iterate over traces in the XML
    for trace in root.findall('.//{http://www.xes-standard.org/}trace'):
        for attr in trace:
            key = attr.attrib.get('key')
            if key:
                unique_keys.add(key)

        for event in trace.findall('./{http://www.xes-standard.org/}event'):
            for attr in event:
                key = attr.attrib.get('key')
                if key:
                    unique_keys.add(key)

    # Generate all possible pairs (relationships) between unique keys
    triples = set()
    unique_keys = list(unique_keys)  # Convert to a list to iterate over pairs

    for i, subject_key in enumerate(unique_keys):
        for j, object_key in enumerate(unique_keys):
            if i != j:  # Avoid self-relation like (key, key)
                subject = f'{base_uri}{subject_key}'
                obj = f'{base_uri}{object_key}'
                triple = (subject, 'ex:subject_object', obj)
                triples.add(triple)

    # Write triples to TTL file
    with open(output_filename, 'w') as f:
        # Write prefix declaration
        f.write('@prefix ex: <http://example.org/> .\n\n')

        for subject, predicate, obj in sorted(triples):
            relation = f'<{subject}> {predicate} <{obj}> .'
            f.write(f'{relation}\n')

def extract_triples_from_ttl(filename):
    triples = set()
    with open(filename, 'r') as f:
        for line in f:
            if line.startswith('<') and ' ex:' in line:
                parts = line.strip().split(' ')
                if len(parts) == 4:  # Adjusted to ensure triple structure
                    subject, predicate, obj, _ = parts
                    triples.add((subject, predicate, obj))
    return triples

def find_common_triples(output_filenames, common_output_filename):
    if not output_filenames:
        return

    common_triples = extract_triples_from_ttl(output_filenames[0])

    for filename in output_filenames[1:]:
        current_triples = extract_triples_from_ttl(filename)
        common_triples &= current_triples

    with open(common_output_filename, 'w') as f:
        # Write prefix declaration
        f.write('@prefix ex: <http://example.org/> .\n\n')

        # Sort the triples before writing
        sorted_triples = sorted(common_triples)

        for triple in sorted_triples:
            f.write(f'{triple[0]} {triple[1]} {triple[2]} .\n')

if __name__ == "__main__":
    input_filenames = ["13small.xes"]
    output_filenames = [f"{filename.split('.')[0]}_relation.ttl" for filename in input_filenames]
    common_output_filename = "bpic13/common_relationsBPIC13.ttl"

    if len(input_filenames) != len(output_filenames):
        raise ValueError("The number of input filenames must match the number of output filenames.")

    for input_filename, output_filename in zip(input_filenames, output_filenames):
        if os.path.exists(input_filename):
            create_ttl_relations(input_filename, output_filename)
        else:
            print(f"File not found: {input_filename}")

    find_common_triples(output_filenames, common_output_filename)


File not found: bpic13/13small.xes


FileNotFoundError: [Errno 2] No such file or directory: 'bpic13/13small_relation.ttl'